In [15]:
## implementation of gravity

import numpy as np

class Vector3:
    """A simple 3D vector class for position, velocity, and acceleration."""
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
    
    def __str__(self):
        return f"({self.x:.2f}, {self.y:.2f}, {self.z:.2f})"


class Entity:
    """An entity in the physics simulation with physical properties."""
    def __init__(self, x=0.0, y=10.0, z=0.0, mass=1.0):
        self.position = Vector3(x, y, z)
        self.velocity = Vector3(0, 0, 0)
        self.acceleration = Vector3(0, 0, 0)
        self.mass = mass
        self.restitution = 0.7  # Bounciness factor (0 = no bounce, 1 = perfect bounce)
        self.grounded = False


class PhysicsSimulation:
    """Physics simulation with gravity and basic collision handling."""
    def __init__(self):
        # Earth's gravity in m/s²
        self.gravity = 9.81
        
        # Simulation settings
        self.time_step = 0.016  # 60fps equivalent in seconds
        self.ground_level = 0.0  # y-coordinate of ground level
        
        # Collection of all entities in the simulation
        self.entities = []
    
    def add_entity(self, entity):
        """Add an entity to the simulation."""
        # Ensure entity is at or above ground level
        if entity.position.y < self.ground_level:
            entity.position.y = self.ground_level
        
        self.entities.append(entity)
        return entity
    
    def create_entity(self, x=0.0, y=10.0, z=0.0, mass=1.0):
        """Create a new entity with physics properties."""
        entity = Entity(x, y, z, mass)
        return self.add_entity(entity)
    
    def apply_force(self, entity, force_x, force_y, force_z):
        """Apply a force to an entity."""
        # F = ma, so a = F/m
        entity.acceleration.x += force_x / entity.mass
        entity.acceleration.y += force_y / entity.mass
        entity.acceleration.z += force_z / entity.mass
        
        # Apply acceleration to velocity
        entity.velocity.x += entity.acceleration.x * self.time_step
        entity.velocity.y += entity.acceleration.y * self.time_step
        entity.velocity.z += entity.acceleration.z * self.time_step
        
        # Reset acceleration
        entity.acceleration.x = 0
        entity.acceleration.y = 0
        entity.acceleration.z = 0
    
    def is_grounded(self, entity):
        """Check if an entity is on the ground."""
        return abs(entity.position.y - self.ground_level) < 0.01 and entity.velocity.y <= 0
    
    def update(self):
        """Apply gravitational force to all entities and update their positions."""
        for entity in self.entities:
            # Only apply gravity if not grounded or if about to leave the ground (positive velocity)
            if not entity.grounded or entity.velocity.y > 0:
                entity.velocity.y -= self.gravity * self.time_step
            
            # Update position based on velocity
            entity.position.x += entity.velocity.x * self.time_step
            entity.position.y += entity.velocity.y * self.time_step
            entity.position.z += entity.velocity.z * self.time_step
            
            # Check for ground collision
            if entity.position.y <= self.ground_level:
                entity.position.y = self.ground_level
                entity.grounded = True
                
                # Bounce if moving downward
                if entity.velocity.y < 0:
                    entity.velocity.y = -entity.velocity.y * entity.restitution
                    
                    # Apply friction to horizontal movement
                    entity.velocity.x *= 0.95
                    entity.velocity.z *= 0.95
                    
                    # Stop very small bounces
                    if abs(entity.velocity.y) < 0.1:
                        entity.velocity.y = 0
            else:
                entity.grounded = False
            



def run_simulation(steps=100, step_size=10):
    """Run the simulation for a specified number of steps."""
    # Create the simulation
    simulation = PhysicsSimulation()
    
    # Create an entity (starting at x=0, y=10, z=0 with mass=1)
    agent = simulation.create_entity()
    
    print(f"Starting simulation with gravity: {simulation.gravity} m/s²")
    print(f"Step 0: Position {agent.position}, "
          f"Velocity: ({agent.velocity.x:.2f}, {agent.velocity.y:.2f}, {agent.velocity.z:.2f}), "
          f"Grounded: {agent.grounded}")
    
    for i in range(1, steps+1):
        simulation.update()
        
        # Log based on specified step size
        if i % step_size == 0:
            print(f"Step {i}: Position {agent.position}, "
                  f"Velocity: ({agent.velocity.x:.2f}, {agent.velocity.y:.2f}, {agent.velocity.z:.2f}), "
                  f"Grounded: {agent.grounded}")
    
    print("Simulation complete")


if __name__ == "__main__":
    run_simulation()

Starting simulation with gravity: 9.81 m/s²
Step 0: Position (0.00, 10.00, 0.00), Velocity: (0.00, 0.00, 0.00), Grounded: False
Step 10: Position (0.00, 9.86, 0.00), Velocity: (0.00, -1.57, 0.00), Grounded: False
Step 20: Position (0.00, 9.47, 0.00), Velocity: (0.00, -3.14, 0.00), Grounded: False
Step 30: Position (0.00, 8.83, 0.00), Velocity: (0.00, -4.71, 0.00), Grounded: False
Step 40: Position (0.00, 7.94, 0.00), Velocity: (0.00, -6.28, 0.00), Grounded: False
Step 50: Position (0.00, 6.80, 0.00), Velocity: (0.00, -7.85, 0.00), Grounded: False
Step 60: Position (0.00, 5.40, 0.00), Velocity: (0.00, -9.42, 0.00), Grounded: False
Step 70: Position (0.00, 3.76, 0.00), Velocity: (0.00, -10.99, 0.00), Grounded: False
Step 80: Position (0.00, 1.86, 0.00), Velocity: (0.00, -12.56, 0.00), Grounded: False
Step 90: Position (0.00, 0.15, 0.00), Velocity: (0.00, 9.62, 0.00), Grounded: False
Step 100: Position (0.00, 1.56, 0.00), Velocity: (0.00, 8.05, 0.00), Grounded: False
Simulation complete


In [17]:
# Assuming the previous code cell with Vector3, Entity, and PhysicsSimulation has been executed

class Agent(Entity):
    """An agent that can move intentionally within the physics simulation."""
    
    def __init__(self, x=0.0, y=0.0, z=0.0, mass=70.0):
        super().__init__(x, y, z, mass)
        
        # Agent properties
        self.max_speed = 5.0  # Maximum horizontal speed in m/s
        self.jump_force = 10.0  # Jump force in m/s (initial velocity)
        self.move_force = 1000.0  # Horizontal movement force
        self.name = "Agent"
        
        # Movement state
        self.move_direction = Vector3(0, 0, 0)  # Normalized direction vector
        self.is_jumping = False
        self.jump_cooldown = 0
        
        # Force grounded state if starting at ground level
        if abs(self.position.y) < 0.01:
            self.position.y = 0.0
            self.grounded = True
        
    def move(self, direction_x, direction_z):
        """Set the movement direction for the agent."""
        # Normalize the direction vector
        magnitude = (direction_x**2 + direction_z**2)**0.5
        if magnitude > 0:
            self.move_direction.x = direction_x / magnitude
            self.move_direction.z = direction_z / magnitude
        else:
            self.move_direction.x = 0
            self.move_direction.z = 0
        
        # Debug info
        print(f"Agent {self.name} movement direction set to ({self.move_direction.x:.2f}, {self.move_direction.z:.2f})")
    
    def jump(self):
        """Make the agent jump if it's on the ground and not in cooldown."""
        if self.grounded and self.jump_cooldown <= 0:
            self.velocity.y = self.jump_force
            self.is_jumping = True
            self.jump_cooldown = 10  # Cooldown frames before can jump again
            # Temporarily set grounded to False immediately to prevent multiple jumps
            self.grounded = False
            print(f"Agent {self.name} jumped with force {self.jump_force}")
            return True
        else:
            # Debug information
            if not self.grounded:
                print(f"Jump failed: Agent {self.name} is not grounded")
            elif self.jump_cooldown > 0:
                print(f"Jump failed: Agent {self.name} is on cooldown ({self.jump_cooldown} steps left)")
            return False
    
    def update(self, simulation):
        """Update agent's state within the simulation context."""
        # Reduce jump cooldown if active
        if self.jump_cooldown > 0:
            self.jump_cooldown -= 1
        
        # Apply movement force if agent is trying to move
        if self.move_direction.x != 0 or self.move_direction.z != 0:
            # Scale force by mass and apply in move direction
            force_x = self.move_direction.x * self.move_force
            force_z = self.move_direction.z * self.move_force
            
            # Apply more force when on ground (better control) than in air
            ground_multiplier = 1.0 if self.grounded else 0.2
            simulation.apply_force(self, 
                                  force_x * ground_multiplier, 
                                  0,  # No vertical force from movement
                                  force_z * ground_multiplier)
        
        # Apply speed limiting
        horizontal_speed = (self.velocity.x**2 + self.velocity.z**2)**0.5
        if horizontal_speed > self.max_speed:
            # Scale down to max speed
            scale = self.max_speed / horizontal_speed
            self.velocity.x *= scale
            self.velocity.z *= scale
        
        # Apply air resistance/drag when not on ground
        if not self.grounded:
            drag_factor = 0.99  # Slight air resistance
            self.velocity.x *= drag_factor
            self.velocity.z *= drag_factor
        # More friction when on ground and not actively moving
        elif abs(self.move_direction.x) < 0.1 and abs(self.move_direction.z) < 0.1:
            # Stronger friction when not trying to move
            ground_friction = 0.85
            self.velocity.x *= ground_friction
            self.velocity.z *= ground_friction
    
    def stop(self):
        """Stop all horizontal movement."""
        self.move_direction.x = 0
        self.move_direction.z = 0
    
    def __str__(self):
        """Return a string representation of the agent."""
        state = "Grounded" if self.grounded else "Airborne"
        return (f"{self.name} at {self.position}, "
                f"Speed: {(self.velocity.x**2 + self.velocity.z**2)**0.5:.2f} m/s, "
                f"State: {state}")


class World:
    """A world containing physics simulation and agents."""
    
    def __init__(self, width=100.0, depth=100.0):
        self.simulation = PhysicsSimulation()
        self.width = width  # X dimension
        self.depth = depth  # Z dimension
        self.agents = []
        
        # World settings
        self.time_step = self.simulation.time_step
        
    def add_agent(self, x=0.0, y=1.0, z=0.0):
        """Add a new agent to the world."""
        # Create the agent with proper initial height
        agent = Agent(x, y, z)
        
        # Add to physics simulation
        self.simulation.add_entity(agent)
        
        # Add to agent list
        self.agents.append(agent)
        return agent
    
    def update(self):
        """Update the world for one time step."""
        # Update all agents
        for agent in self.agents:
            agent.update(self.simulation)
        
        # Update physics
        self.simulation.update()
        
        # Enforce world boundaries for all agents
        for agent in self.agents:
            # X boundaries
            if agent.position.x < 0:
                agent.position.x = 0
                agent.velocity.x = 0
            elif agent.position.x > self.width:
                agent.position.x = self.width
                agent.velocity.x = 0
                
            # Z boundaries
            if agent.position.z < 0:
                agent.position.z = 0
                agent.velocity.z = 0
            elif agent.position.z > self.depth:
                agent.position.z = self.depth
                agent.velocity.z = 0

In [11]:
# Simple test to verify gravity works correctly
import time

# Create a simple entity
entity = Entity(0.0, 10.0, 0.0)  # Starting 10 units above ground
print(f"Created entity at y={entity.position.y} with velocity.y={entity.velocity.y}")

# Create physics simulation
simulation = PhysicsSimulation()
simulation.add_entity(entity)

# Run simulation manually step by step to see exactly what happens
print("\nRunning gravity simulation:")
for i in range(100):
    # Before update
    old_y = entity.position.y
    old_velocity = entity.velocity.y
    
    # Run physics update
    simulation.update()
    
    # Only print every 5 steps to not flood output
    if i % 5 == 0:
        print(f"Step {i}: Position y: {old_y:.2f} → {entity.position.y:.2f}, "
              f"Velocity y: {old_velocity:.2f} → {entity.velocity.y:.2f}, "
              f"Grounded: {entity.grounded}")
    
    # Stop if on ground and not moving
    if entity.grounded and abs(entity.velocity.y) < 0.01:
        print(f"Entity has come to rest on ground at step {i}")
        break

Created entity at y=10.0 with velocity.y=0.0

Running gravity simulation:
Step 0: Position y: 10.00 → 10.00, Velocity y: 0.00 → -0.16, Grounded: False
Step 5: Position y: 9.96 → 9.95, Velocity y: -0.78 → -0.94, Grounded: False
Step 10: Position y: 9.86 → 9.83, Velocity y: -1.57 → -1.73, Grounded: False
Step 15: Position y: 9.70 → 9.66, Velocity y: -2.35 → -2.51, Grounded: False
Step 20: Position y: 9.47 → 9.42, Velocity y: -3.14 → -3.30, Grounded: False
Step 25: Position y: 9.18 → 9.12, Velocity y: -3.92 → -4.08, Grounded: False
Step 30: Position y: 8.83 → 8.75, Velocity y: -4.71 → -4.87, Grounded: False
Step 35: Position y: 8.42 → 8.33, Velocity y: -5.49 → -5.65, Grounded: False
Step 40: Position y: 7.94 → 7.84, Velocity y: -6.28 → -6.44, Grounded: False
Step 45: Position y: 7.40 → 7.29, Velocity y: -7.06 → -7.22, Grounded: False
Step 50: Position y: 6.80 → 6.67, Velocity y: -7.85 → -8.00, Grounded: False
Step 55: Position y: 6.13 → 5.99, Velocity y: -8.63 → -8.79, Grounded: False
Ste

In [12]:
# Looking at the issue with movement, let's make a direct test
# with focus only on horizontal movement

# First, let's fix the agent.move method
def move_fixed(agent, direction_x, direction_z):
    """Corrected movement function"""
    # Normalize the direction vector
    magnitude = (direction_x**2 + direction_z**2)**0.5
    if magnitude > 0:
        # Here's a key fix - storing direction properly
        agent.move_direction = Vector3(
            direction_x / magnitude,
            0,  # No vertical component to movement direction
            direction_z / magnitude
        )
    else:
        agent.move_direction = Vector3(0, 0, 0)
    
    print(f"Agent direction set to: x={agent.move_direction.x:.2f}, z={agent.move_direction.z:.2f}")


# Create a new test world
world = World(width=50.0, depth=50.0)
agent = world.add_agent(x=25.0, y=0.0, z=25.0)
agent.name = "MovementTest"

# Replace the agent's move method with our fixed version
agent.move = lambda dx, dz: move_fixed(agent, dx, dz)

print(f"Testing movement with fixed method")
print(f"Initial position: {agent.position}")

# Try moving north (negative z)
print("\nMoving NORTH (negative z)...")
agent.move(0, -1)

for i in range(20):
    world.update()
    if i % 5 == 0:
        print(f"Step {i}: Position={agent.position}, Speed={agent.velocity.x:.2f},{agent.velocity.z:.2f}")

# Try moving east (positive x)
print("\nMoving EAST (positive x)...")
agent.move(1, 0)

for i in range(20):
    world.update()
    if i % 5 == 0:
        print(f"Step {i}: Position={agent.position}, Speed={agent.velocity.x:.2f},{agent.velocity.z:.2f}")

# Try moving diagonally
print("\nMoving SOUTHEAST (diagonal)...")
agent.move(1, 1)

for i in range(20):
    world.update()
    if i % 5 == 0:
        print(f"Step {i}: Position={agent.position}, Speed={agent.velocity.x:.2f},{agent.velocity.z:.2f}")

print(f"\nFinal position: {agent.position}")

Testing movement with fixed method
Initial position: (25.00, 0.00, 25.00)

Moving NORTH (negative z)...
Agent direction set to: x=0.00, z=-1.00
Step 0: Position=(25.00, 0.00, 25.00), Speed=0.00,-0.23
Step 5: Position=(25.00, 0.00, 24.92), Speed=0.00,-1.37
Step 10: Position=(25.00, 0.00, 24.76), Speed=0.00,-2.51
Step 15: Position=(25.00, 0.00, 24.50), Speed=0.00,-3.66

Moving EAST (positive x)...
Agent direction set to: x=1.00, z=0.00
Step 0: Position=(25.00, 0.00, 24.16), Speed=0.23,-4.57
Step 5: Position=(25.08, 0.00, 23.79), Speed=1.37,-4.57
Step 10: Position=(25.24, 0.00, 23.43), Speed=2.41,-4.38
Step 15: Position=(25.47, 0.00, 23.11), Speed=3.18,-3.86

Moving SOUTHEAST (diagonal)...
Agent direction set to: x=0.71, z=0.71
Step 0: Position=(25.75, 0.00, 22.83), Speed=3.81,-3.24
Step 5: Position=(26.09, 0.00, 22.61), Speed=4.43,-2.32
Step 10: Position=(26.46, 0.00, 22.47), Speed=4.81,-1.35
Step 15: Position=(26.86, 0.00, 22.41), Speed=4.98,-0.44

Final position: (27.18, 0.00, 22.41)


In [13]:
# Completely redefine the physics from scratch to fix the gravity issue

class Vector3:
    """A simple 3D vector class for position, velocity, and acceleration."""
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
    
    def __str__(self):
        return f"({self.x:.2f}, {self.y:.2f}, {self.z:.2f})"


class Entity:
    """An entity in the physics simulation with physical properties."""
    def __init__(self, x=0.0, y=10.0, z=0.0, mass=1.0):
        self.position = Vector3(x, y, z)
        self.velocity = Vector3(0, 0, 0)
        self.acceleration = Vector3(0, 0, 0)
        self.mass = mass
        self.restitution = 0.7  # Bounciness factor (0 = no bounce, 1 = perfect bounce)
        self.grounded = False


class PhysicsSimulation:
    """Physics simulation with gravity and basic collision handling."""
    def __init__(self):
        # Earth's gravity in m/s²
        self.gravity = 9.81
        
        # Simulation settings
        self.time_step = 0.016  # 60fps equivalent in seconds
        self.ground_level = 0.0  # y-coordinate of ground level
        
        # Collection of all entities in the simulation
        self.entities = []
    
    def add_entity(self, entity):
        """Add an entity to the simulation."""
        # Ensure entity is at or above ground level
        if entity.position.y < self.ground_level:
            entity.position.y = self.ground_level
            entity.grounded = True
        
        self.entities.append(entity)
        return entity
    
    def apply_force(self, entity, force_x, force_y, force_z):
        """Apply a force to an entity."""
        # F = ma, so a = F/m
        # Add to acceleration
        entity.acceleration.x += force_x / entity.mass
        entity.acceleration.y += force_y / entity.mass
        entity.acceleration.z += force_z / entity.mass
    
    def update(self):
        """Update the physics simulation for one time step."""
        for entity in self.entities:
            # Reset acceleration at the start of each update
            entity.acceleration.x = 0
            entity.acceleration.y = 0
            entity.acceleration.z = 0
            
            # Apply gravity force (only if not grounded)
            if not entity.grounded:
                entity.acceleration.y -= self.gravity
            
            # Apply acceleration to velocity
            entity.velocity.x += entity.acceleration.x * self.time_step
            entity.velocity.y += entity.acceleration.y * self.time_step
            entity.velocity.z += entity.acceleration.z * self.time_step
            
            # Apply velocity to position
            entity.position.x += entity.velocity.x * self.time_step
            entity.position.y += entity.velocity.y * self.time_step
            entity.position.z += entity.velocity.z * self.time_step
            
            # Handle ground collision
            if entity.position.y <= self.ground_level:
                entity.position.y = self.ground_level
                
                # If moving downward, handle bounce or stop
                if entity.velocity.y < 0:
                    bounce_velocity = -entity.velocity.y * entity.restitution
                    
                    # If bounce is significant, bounce; otherwise stop
                    if bounce_velocity > 0.1:
                        entity.velocity.y = bounce_velocity
                    else:
                        entity.velocity.y = 0
                    
                    # Apply horizontal friction on impact
                    entity.velocity.x *= 0.9
                    entity.velocity.z *= 0.9
                
                # Update grounded state
                entity.grounded = True
            elif entity.velocity.y != 0:
                # In the air and moving
                entity.grounded = False


# Test the fixed physics
def test_gravity():
    print("Testing gravity with fixed physics:")
    
    # Create simulation
    simulation = PhysicsSimulation()
    
    # Create test entity 10 units above ground
    entity = Entity(0, 10, 0)
    simulation.add_entity(entity)
    
    # Run simulation
    for i in range(100):
        old_y = entity.position.y
        old_vy = entity.velocity.y
        
        simulation.update()
        
        # Print details every 10 steps
        if i % 10 == 0:
            print(f"Step {i}: y={entity.position.y:.2f} (was {old_y:.2f}), "
                 f"vy={entity.velocity.y:.2f} (was {old_vy:.2f}), "
                 f"Grounded={entity.grounded}")
        
        # Stop if on ground and not moving
        if entity.grounded and abs(entity.velocity.y) < 0.01:
            print(f"Entity reached ground and stopped at step {i}")
            break

# Run the test
test_gravity()

Testing gravity with fixed physics:
Step 0: y=10.00 (was 10.00), vy=-0.16 (was 0.00), Grounded=False
Step 10: y=9.83 (was 9.86), vy=-1.73 (was -1.57), Grounded=False
Step 20: y=9.42 (was 9.47), vy=-3.30 (was -3.14), Grounded=False
Step 30: y=8.75 (was 8.83), vy=-4.87 (was -4.71), Grounded=False
Step 40: y=7.84 (was 7.94), vy=-6.44 (was -6.28), Grounded=False
Step 50: y=6.67 (was 6.80), vy=-8.00 (was -7.85), Grounded=False
Step 60: y=5.25 (was 5.40), vy=-9.57 (was -9.42), Grounded=False
Step 70: y=3.58 (was 3.76), vy=-11.14 (was -10.99), Grounded=False
Step 80: y=1.66 (was 1.86), vy=-12.71 (was -12.56), Grounded=False
Step 90: y=0.31 (was 0.16), vy=9.62 (was 9.78), Grounded=False


In [14]:
class Agent(Entity):
    """An agent that can move intentionally within the physics simulation."""
    
    def __init__(self, x=0.0, y=0.0, z=0.0, mass=70.0):
        super().__init__(x, y, z, mass)
        
        # Agent properties
        self.max_speed = 5.0  # Maximum horizontal speed in m/s
        self.jump_force = 10.0  # Jump force in m/s (initial velocity)
        self.move_force = 1000.0  # Horizontal movement force
        self.name = "Agent"
        
        # Movement state - IMPORTANT FIX: store as separate values, not as Vector3
        self.move_direction_x = 0.0
        self.move_direction_z = 0.0
        self.is_jumping = False
        self.jump_cooldown = 0
        
        # Force grounded state if starting at ground level
        if abs(self.position.y) < 0.01:
            self.position.y = 0.0
            self.grounded = True
        
    def move(self, direction_x, direction_z):
        """Set the movement direction for the agent."""
        # Calculate magnitude
        magnitude = (direction_x**2 + direction_z**2)**0.5
        
        # Normalize and store direction components
        if magnitude > 0:
            self.move_direction_x = direction_x / magnitude
            self.move_direction_z = direction_z / magnitude
        else:
            self.move_direction_x = 0
            self.move_direction_z = 0
        
        print(f"Agent {self.name} movement direction set to ({self.move_direction_x:.2f}, {self.move_direction_z:.2f})")
    
    def jump(self):
        """Make the agent jump if it's on the ground and not in cooldown."""
        if self.grounded and self.jump_cooldown <= 0:
            # Set upward velocity
            self.velocity.y = self.jump_force
            
            # Update state
            self.is_jumping = True
            self.jump_cooldown = 10  # Cooldown frames
            self.grounded = False  # No longer grounded
            
            print(f"Agent {self.name} jumped with force {self.jump_force}")
            return True
        else:
            # Debug information
            if not self.grounded:
                print(f"Jump failed: Agent {self.name} is not grounded")
            elif self.jump_cooldown > 0:
                print(f"Jump failed: Agent {self.name} is on cooldown ({self.jump_cooldown} steps left)")
            return False
    
    def update(self, simulation):
        """Update agent's state within the simulation context."""
        # Reduce jump cooldown if active
        if self.jump_cooldown > 0:
            self.jump_cooldown -= 1
        
        # Apply movement force if agent is trying to move
        if self.move_direction_x != 0 or self.move_direction_z != 0:
            # Calculate force in each direction
            force_x = self.move_direction_x * self.move_force
            force_z = self.move_direction_z * self.move_force
            
            # Apply more force when on ground (better control) than in air
            ground_multiplier = 1.0 if self.grounded else 0.2
            
            # Apply forces through simulation
            simulation.apply_force(self, 
                                  force_x * ground_multiplier, 
                                  0,  # No vertical force from movement
                                  force_z * ground_multiplier)
        
        # Apply speed limiting for horizontal movement only
        horizontal_speed = (self.velocity.x**2 + self.velocity.z**2)**0.5
        if horizontal_speed > self.max_speed:
            # Scale down to max speed
            scale = self.max_speed / horizontal_speed
            self.velocity.x *= scale
            self.velocity.z *= scale
        
        # Apply air resistance/drag when not on ground
        if not self.grounded:
            drag_factor = 0.99  # Slight air resistance
            self.velocity.x *= drag_factor
            self.velocity.z *= drag_factor
        # More friction when on ground and not actively moving
        elif abs(self.move_direction_x) < 0.1 and abs(self.move_direction_z) < 0.1:
            # Stronger friction when not trying to move
            ground_friction = 0.85
            self.velocity.x *= ground_friction
            self.velocity.z *= ground_friction
    
    def stop(self):
        """Stop all horizontal movement."""
        self.move_direction_x = 0
        self.move_direction_z = 0
        print(f"Agent {self.name} stopped movement")
    
    def __str__(self):
        """Return a string representation of the agent."""
        state = "Grounded" if self.grounded else "Airborne"
        speed = (self.velocity.x**2 + self.velocity.z**2)**0.5
        return (f"{self.name} at {self.position}, "
                f"Speed: {speed:.2f} m/s, "
                f"State: {state}")


# Test the fixed Agent class
def test_fixed_agent():
    # Create world and agent
    world = World(width=50.0, depth=50.0)
    agent = world.add_agent(x=25.0, y=0.0, z=25.0)
    agent.name = "FixedTest"
    
    print(f"Created agent at {agent.position}")
    
    # Test 1: North movement
    print("\nTest 1: Moving North")
    agent.move(0, -1)  # North is negative Z
    
    for i in range(20):
        world.update()
        if i % 5 == 0:
            print(f"Step {i}: {agent}")
    
    # Test 2: East movement
    print("\nTest 2: Moving East")
    agent.move(1, 0)  # East is positive X
    
    for i in range(20):
        world.update()
        if i % 5 == 0:
            print(f"Step {i}: {agent}")
    
    # Test 3: Jump
    print("\nTest 3: Stopping and Jumping")
    agent.stop()
    jump_result = agent.jump()
    
    for i in range(20):
        world.update()
        if i % 5 == 0:
            print(f"Step {i}: {agent}")
    
    print(f"Final position: {agent.position}")

# Run the test
test_fixed_agent()

Created agent at (25.00, 0.00, 25.00)

Test 1: Moving North
Agent FixedTest movement direction set to (0.00, -1.00)
Step 0: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded
Step 5: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded
Step 10: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded
Step 15: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded

Test 2: Moving East
Agent FixedTest movement direction set to (1.00, 0.00)
Step 0: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded
Step 5: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded
Step 10: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded
Step 15: FixedTest at (25.00, 0.00, 25.00), Speed: 0.00 m/s, State: Grounded

Test 3: Stopping and Jumping
Agent FixedTest stopped movement
Agent FixedTest jumped with force 10.0
Step 0: FixedTest at (25.00, 0.16, 25.00), Speed: 0.00 m/s, State: Airborne
Step 5: FixedTest 

In [21]:
import random
import math

class RoamingAgent(Agent):
    """An agent that can autonomously roam around the world."""
    
    def __init__(self, x=0.0, y=0.0, z=0.0, mass=70.0):
        super().__init__(x, y, z, mass)
        
        # Roaming parameters
        self.roam_radius = 20.0  # How far the agent will roam
        self.direction_change_cooldown = 0
        self.max_direction_time = 120  # Maximum time to move in one direction
        self.rest_time = 0  # Counter for resting between movements
        self.max_rest_time = 60  # Maximum time to rest
        self.home_position = Vector3(x, y, z)  # Starting position as "home"
        self.jump_probability = 0.01  # Chance to jump while roaming
        
    def update(self, simulation):
        """Update agent with autonomous behavior."""
        # Decrease cooldowns
        if self.direction_change_cooldown > 0:
            self.direction_change_cooldown -= 1
        if self.rest_time > 0:
            self.rest_time -= 1
            
        # Decision making
        if self.direction_change_cooldown <= 0:
            # Time to make a new decision
            if self.rest_time <= 0:
                # Not resting, so decide: move or rest?
                if random.random() < 0.8:  # 80% chance to move
                    # Choose a random direction
                    angle = random.uniform(0, 2*math.pi)
                    self.move(math.cos(angle), math.sin(angle))
                    
                    # Set cooldown for next direction change
                    self.direction_change_cooldown = random.randint(30, self.max_direction_time)
                else:
                    # Decide to rest
                    self.stop()
                    self.rest_time = random.randint(20, self.max_rest_time)
            else:
                # Currently resting
                self.stop()
        
        # Random jumping while moving
        # Adapt this to work with either version of the Agent class
        is_moving = False
        
        # Check if agent is moving by examining the move_direction
        try:
            # For the version using move_direction_x and move_direction_z
            is_moving = (self.move_direction_x != 0 or self.move_direction_z != 0)
        except AttributeError:
            # For the version using move_direction as a Vector3
            is_moving = (self.move_direction.x != 0 or self.move_direction.z != 0)
        
        if self.grounded and random.random() < self.jump_probability and is_moving:
            self.jump()
        
        # Tendency to return home if too far away
        distance_from_home = ((self.position.x - self.home_position.x)**2 + 
                             (self.position.z - self.home_position.z)**2)**0.5
        
        if distance_from_home > self.roam_radius and self.direction_change_cooldown <= 0:
            # Head back toward home
            direction_x = self.home_position.x - self.position.x
            direction_z = self.home_position.z - self.position.z
            magnitude = (direction_x**2 + direction_z**2)**0.5
            if magnitude > 0:
                self.move(direction_x/magnitude, direction_z/magnitude)
            self.direction_change_cooldown = 60  # Keep this direction for a while
            
        # Call the parent update method to handle physics
        super().update(simulation)

In [22]:
import random
import math
import time

def create_roaming_world(num_agents=5, world_size=100.0, update_steps=1000):
    """Create a world with multiple roaming agents and run the simulation."""
    # Create world
    world = World(width=world_size, depth=world_size)
    
    # Create agents at random positions
    for i in range(num_agents):
        x = random.uniform(10, world_size-10)
        z = random.uniform(10, world_size-10)
        agent = RoamingAgent(x=x, y=0.0, z=z)
        agent.name = f"Agent-{i+1}"
        
        # Add to world
        world.agents.append(agent)
        world.simulation.add_entity(agent)
        
        print(f"Created {agent.name} at {agent.position}")
    
    # Run simulation for specified number of steps
    print(f"\nRunning simulation with {num_agents} roaming agents...")
    
    for step in range(update_steps):
        # Update world
        world.update()
        
        # Print information periodically
        if step % 100 == 0:
            print(f"\nStep {step}:")
            for agent in world.agents:
                print(f"  {agent}")
        
        # Add a small delay to make the updates visible if this were visualized
        time.sleep(0.01)
    
    print("\nSimulation complete!")

# Run the simulation
create_roaming_world(num_agents=3, update_steps=500)

Created Agent-1 at (20.96, 0.00, 56.98)
Created Agent-2 at (83.75, 0.00, 51.05)
Created Agent-3 at (87.18, 0.00, 71.78)

Running simulation with 3 roaming agents...
Agent Agent-1 movement direction set to (-1.00, -0.06)
Agent Agent-2 movement direction set to (-0.94, 0.33)
Agent Agent-3 movement direction set to (-0.87, -0.49)

Step 0:
  Agent-1 at (20.96, 0.00, 56.98), Speed: 0.23 m/s, State: Grounded
  Agent-2 at (83.75, 0.00, 51.05), Speed: 0.23 m/s, State: Grounded
  Agent-3 at (87.18, 0.00, 71.78), Speed: 0.23 m/s, State: Grounded
Agent Agent-1 jumped with force 10.0
Agent Agent-3 movement direction set to (0.85, 0.53)
Agent Agent-1 movement direction set to (-0.95, -0.32)
Agent Agent-2 movement direction set to (-0.84, -0.55)

Step 100:
  Agent-1 at (13.91, 5.01, 56.42), Speed: 4.73 m/s, State: Airborne
  Agent-2 at (80.84, 0.00, 52.04), Speed: 0.91 m/s, State: Grounded
  Agent-3 at (84.34, 0.00, 70.40), Speed: 5.00 m/s, State: Grounded
Agent Agent-1 movement direction set to (0.

In [23]:
def visualize_world(world, cell_size=5):
    """Create a text-based visualization of the world."""
    width_cells = int(world.width / cell_size)
    depth_cells = int(world.depth / cell_size)
    
    # Create empty grid
    grid = [[' ' for _ in range(width_cells)] for _ in range(depth_cells)]
    
    # Mark boundaries
    for x in range(width_cells):
        grid[0][x] = '#'
        grid[depth_cells-1][x] = '#'
    for z in range(depth_cells):
        grid[z][0] = '#'
        grid[z][width_cells-1] = '#'
    
    # Place agents
    for agent in world.agents:
        grid_x = min(width_cells-1, max(0, int(agent.position.x / cell_size)))
        grid_z = min(depth_cells-1, max(0, int(agent.position.z / cell_size)))
        grid[grid_z][grid_x] = 'A'
    
    # Print the grid
    print("\nWorld visualization:")
    for row in grid:
        print(''.join(row))
    print("")

In [24]:
import numpy as np

class NeuralNetwork:
    """Simple feedforward neural network for agent decision making."""
    
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize with random weights
        self.weights_input_hidden = np.random.randn(input_size, hidden_size) * 0.1
        self.weights_hidden_output = np.random.randn(hidden_size, output_size) * 0.1
        
        # Biases
        self.bias_hidden = np.zeros((1, hidden_size))
        self.bias_output = np.zeros((1, output_size))
    
    def forward(self, inputs):
        """Forward pass through the network."""
        # Convert inputs to numpy array if not already
        inputs = np.array(inputs).reshape(1, -1)
        
        # Hidden layer with ReLU activation
        self.hidden = np.dot(inputs, self.weights_input_hidden) + self.bias_hidden
        self.hidden_activated = np.maximum(0, self.hidden)  # ReLU
        
        # Output layer with softmax activation for action probabilities
        self.output = np.dot(self.hidden_activated, self.weights_hidden_output) + self.bias_output
        self.output_activated = self._softmax(self.output)
        
        return self.output_activated
    
    def _softmax(self, x):
        """Compute softmax values for each set of scores in x."""
        exp_x = np.exp(x - np.max(x))  # Subtract max for numerical stability
        return exp_x / exp_x.sum(axis=1, keepdims=True)
    
    def mutate(self, mutation_rate=0.1, mutation_scale=0.2):
        """Randomly mutate weights and biases for evolution."""
        # Mutate weights
        mutation_mask = np.random.random(self.weights_input_hidden.shape) < mutation_rate
        self.weights_input_hidden += mutation_mask * np.random.randn(*self.weights_input_hidden.shape) * mutation_scale
        
        mutation_mask = np.random.random(self.weights_hidden_output.shape) < mutation_rate
        self.weights_hidden_output += mutation_mask * np.random.randn(*self.weights_hidden_output.shape) * mutation_scale
        
        # Mutate biases
        mutation_mask = np.random.random(self.bias_hidden.shape) < mutation_rate
        self.bias_hidden += mutation_mask * np.random.randn(*self.bias_hidden.shape) * mutation_scale
        
        mutation_mask = np.random.random(self.bias_output.shape) < mutation_rate
        self.bias_output += mutation_mask * np.random.randn(*self.bias_output.shape) * mutation_scale

In [25]:
class NeuralAgent(Agent):
    """An agent controlled by a neural network."""
    
    def __init__(self, x=0.0, y=0.0, z=0.0, mass=70.0):
        super().__init__(x, y, z, mass)
        
        # Agent properties
        self.name = f"Neural-{id(self) % 1000}"
        self.decision_cooldown = 0
        self.decision_interval = 10  # Make decisions every N steps
        self.home_position = Vector3(x, y, z)
        
        # Neural network setup
        self.input_size = 6  # Position x,z, velocity x,z, distance from home, grounded
        self.hidden_size = 8
        self.output_size = 5  # N, E, S, W, Jump
        self.brain = NeuralNetwork(self.input_size, self.hidden_size, self.output_size)
        
        # Performance metrics
        self.distance_traveled = 0.0
        self.jumps_performed = 0
        self.fitness = 0.0
        self.last_position = Vector3(x, y, z)
    
    def update(self, simulation):
        """Update agent using neural network decisions."""
        # Reduce decision cooldown
        if self.decision_cooldown > 0:
            self.decision_cooldown -= 1
        
        # Make decision using neural network
        if self.decision_cooldown <= 0:
            self._make_decision()
            self.decision_cooldown = self.decision_interval
        
        # Calculate distance traveled since last update
        distance_moved = ((self.position.x - self.last_position.x)**2 + 
                          (self.position.z - self.last_position.z)**2)**0.5
        self.distance_traveled += distance_moved
        
        # Update last position
        self.last_position.x = self.position.x
        self.last_position.y = self.position.y
        self.last_position.z = self.position.z
        
        # Call parent update for physics
        super().update(simulation)
    
    def _make_decision(self):
        """Use neural network to decide actions."""
        # Prepare inputs for the neural network
        distance_from_home = ((self.position.x - self.home_position.x)**2 + 
                             (self.position.z - self.home_position.z)**2)**0.5
        
        inputs = [
            self.position.x / 50.0,  # Normalized position
            self.position.z / 50.0,
            self.velocity.x / self.max_speed,  # Normalized velocity
            self.velocity.z / self.max_speed,
            distance_from_home / 30.0,  # Normalized distance from home
            1.0 if self.grounded else 0.0  # Grounded state
        ]
        
        # Get network output
        output = self.brain.forward(inputs)[0]  # [0] to get the first (and only) row
        
        # Interpret output as actions
        action_idx = np.argmax(output)
        
        # Apply action
        if action_idx == 0:  # North
            self.move(0, -1)
        elif action_idx == 1:  # East
            self.move(1, 0)
        elif action_idx == 2:  # South
            self.move(0, 1)
        elif action_idx == 3:  # West
            self.move(-1, 0)
        elif action_idx == 4 and self.grounded:  # Jump (only if grounded)
            if self.jump():
                self.jumps_performed += 1
    
    def calculate_fitness(self):
        """Calculate fitness score for evolution."""
        # Simple fitness function based on distance traveled and jumps
        self.fitness = self.distance_traveled + (self.jumps_performed * 2.0)
        return self.fitness
    
    def create_offspring(self, partner=None):
        """Create a new agent with potentially mutated brain."""
        # Create a new agent at the same position
        child = NeuralAgent(self.position.x, self.position.y, self.position.z)
        
        # Copy brain from parent with mutations
        if partner:
            # Crossover between two parents (simple averaging for demonstration)
            child.brain.weights_input_hidden = (self.brain.weights_input_hidden + 
                                               partner.brain.weights_input_hidden) / 2
            child.brain.weights_hidden_output = (self.brain.weights_hidden_output + 
                                                partner.brain.weights_hidden_output) / 2
            child.brain.bias_hidden = (self.brain.bias_hidden + partner.brain.bias_hidden) / 2
            child.brain.bias_output = (self.brain.bias_output + partner.brain.bias_output) / 2
        else:
            # Copy from this parent
            child.brain.weights_input_hidden = self.brain.weights_input_hidden.copy()
            child.brain.weights_hidden_output = self.brain.weights_hidden_output.copy()
            child.brain.bias_hidden = self.brain.bias_hidden.copy()
            child.brain.bias_output = self.brain.bias_output.copy()
        
        # Apply mutation
        child.brain.mutate()
        
        return child

In [26]:
class EvolutionSimulation:
    """Simulation that evolves neural network agents over generations."""
    
    def __init__(self, population_size=10, world_size=100.0):
        self.population_size = population_size
        self.world_size = world_size
        self.generation = 0
        self.best_fitness = 0
        self.world = None
        
    def initialize(self):
        """Initialize the first generation."""
        self.generation = 1
        self.world = World(width=self.world_size, depth=self.world_size)
        
        # Create initial population
        for i in range(self.population_size):
            x = random.uniform(10, self.world_size-10)
            z = random.uniform(10, self.world_size-10)
            
            # Create neural agent
            agent = NeuralAgent(x, 0.0, z)
            
            # Add to world
            self.world.simulation.add_entity(agent)
            self.world.agents.append(agent)
        
        print(f"Generation {self.generation} initialized with {len(self.world.agents)} agents")
    
    def run_generation(self, steps=500):
        """Run the current generation for a specified number of steps."""
        print(f"Running generation {self.generation} for {steps} steps...")
        
        for step in range(steps):
            self.world.update()
            
            # Periodically report progress
            if step % 100 == 0:
                print(f"Generation {self.generation}, Step {step}")
        
        # Calculate fitness for all agents
        fitnesses = []
        for agent in self.world.agents:
            fitness = agent.calculate_fitness()
            fitnesses.append((agent, fitness))
            print(f"Agent {agent.name}: Fitness = {fitness:.2f}")
        
        # Sort by fitness (descending)
        fitnesses.sort(key=lambda x: x[1], reverse=True)
        
        # Record best fitness
        if fitnesses[0][1] > self.best_fitness:
            self.best_fitness = fitnesses[0][1]
        
        print(f"Generation {self.generation} complete")
        print(f"Best fitness: {fitnesses[0][1]:.2f}")
        print(f"Best fitness ever: {self.best_fitness:.2f}")
        
        return fitnesses
    
    def create_next_generation(self, fitnesses):
        """Create the next generation through selection and reproduction."""
        self.generation += 1
        
        # Keep the top performers
        elite_count = max(1, self.population_size // 5)
        
        # Create new world
        new_world = World(width=self.world_size, depth=self.world_size)
        
        # Add elites directly
        for i in range(elite_count):
            if i < len(fitnesses):
                elite_agent = fitnesses[i][0]
                new_world.simulation.add_entity(elite_agent)
                new_world.agents.append(elite_agent)
                
                # Reset position and stats
                elite_agent.position.x = random.uniform(10, self.world_size-10)
                elite_agent.position.z = random.uniform(10, self.world_size-10)
                elite_agent.position.y = 0.0
                elite_agent.velocity.x = 0.0
                elite_agent.velocity.y = 0.0
                elite_agent.velocity.z = 0.0
                elite_agent.home_position = Vector3(elite_agent.position.x, 
                                                   elite_agent.position.y, 
                                                   elite_agent.position.z)
                elite_agent.distance_traveled = 0.0
                elite_agent.jumps_performed = 0
                elite_agent.fitness = 0.0
        
        # Create offspring from the top half
        parents = [agent for agent, _ in fitnesses[:self.population_size//2]]
        
        # Fill the rest of the population
        while len(new_world.agents) < self.population_size:
            # Select two random parents
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            
            # Create child with crossover and mutation
            child = parent1.create_offspring(parent2)
            
            # Set random position
            child.position.x = random.uniform(10, self.world_size-10)
            child.position.z = random.uniform(10, self.world_size-10)
            child.position.y = 0.0
            child.home_position = Vector3(child.position.x, child.position.y, child.position.z)
            
            # Add to new world
            new_world.simulation.add_entity(child)
            new_world.agents.append(child)
        
        # Replace world
        self.world = new_world
        
        print(f"Generation {self.generation} created with {len(self.world.agents)} agents")

In [27]:
def run_evolution(generations=10, steps_per_generation=500, population_size=15):
    """Run the neural network evolution simulation."""
    sim = EvolutionSimulation(population_size=population_size)
    sim.initialize()
    
    for gen in range(generations):
        fitnesses = sim.run_generation(steps=steps_per_generation)
        sim.create_next_generation(fitnesses)
        
        # Optional: Visualize best agent's brain
        best_agent = fitnesses[0][0]
        print(f"Best agent in gen {gen+1}: {best_agent.name}, " +
              f"Distance: {best_agent.distance_traveled:.2f}, " +
              f"Jumps: {best_agent.jumps_performed}")
    
    print("\nEvolution complete!")
    print(f"Best fitness achieved: {sim.best_fitness:.2f}")
    
    return sim

# Run the evolution
evolution_sim = run_evolution(generations=5, steps_per_generation=300)

Generation 1 initialized with 15 agents
Running generation 1 for 300 steps...
Agent Neural-920 movement direction set to (0.00, -1.00)
Agent Neural-8 movement direction set to (1.00, 0.00)
Agent Neural-696 jumped with force 10.0
Agent Neural-280 movement direction set to (-1.00, 0.00)
Agent Neural-512 movement direction set to (1.00, 0.00)
Agent Neural-472 movement direction set to (1.00, 0.00)
Agent Neural-80 movement direction set to (0.00, 1.00)
Agent Neural-232 movement direction set to (1.00, 0.00)
Agent Neural-240 movement direction set to (-1.00, 0.00)
Agent Neural-520 movement direction set to (0.00, 1.00)
Agent Neural-824 movement direction set to (0.00, -1.00)
Agent Neural-200 movement direction set to (0.00, -1.00)
Agent Neural-408 movement direction set to (0.00, -1.00)
Agent Neural-376 movement direction set to (1.00, 0.00)
Agent Neural-328 movement direction set to (1.00, 0.00)
Generation 1, Step 0
Agent Neural-920 movement direction set to (0.00, -1.00)
Agent Neural-8 mo

In [29]:
class Obstacle:
    """An obstacle in the world that agents cannot pass through."""
    
    def __init__(self, x, y, z, width, depth):
        self.x = x  # Left edge X position
        self.y = y  # Height position (typically 0 for ground level)
        self.z = z  # Front edge Z position
        self.width = width  # Size along X axis
        self.depth = depth  # Size along Z axis
    
    def contains_point(self, x, z):
        """Check if a point is inside this obstacle."""
        return (self.x <= x <= self.x + self.width and 
                self.z <= z <= self.z + self.depth)

class World:
    """A world containing physics simulation, agents, and obstacles."""
    
    def __init__(self, width=100.0, depth=100.0):
        self.simulation = PhysicsSimulation()
        self.width = width  # X dimension
        self.depth = depth  # Z dimension
        self.agents = []
        self.obstacles = []  # New list to store obstacles
        
        # World settings
        self.time_step = self.simulation.time_step
    
    def add_obstacle(self, x, y, z, width, depth):
        """Add a new obstacle to the world."""
        obstacle = Obstacle(x, y, z, width, depth)
        self.obstacles.append(obstacle)
        return obstacle
    
    def is_position_valid(self, x, z):
        """Check if a position is valid (not inside any obstacle)."""
        # Check world boundaries
        if x < 0 or x > self.width or z < 0 or z > self.depth:
            return False
        
        # Check if inside any obstacle
        for obstacle in self.obstacles:
            if obstacle.contains_point(x, z):
                return False
        
        return True

def update(self):
    """Update the world for one time step."""
    # Update all agents
    for agent in self.agents:
        agent.update(self.simulation)
    
    # Update physics
    self.simulation.update()
    
    # Handle collisions with obstacles and world boundaries
    for agent in self.agents:
        # Store current position for potential rollback
        old_x = agent.position.x
        old_z = agent.position.z
        
        # X boundaries
        if agent.position.x < 0:
            agent.position.x = 0
            agent.velocity.x = 0
        elif agent.position.x > self.width:
            agent.position.x = self.width
            agent.velocity.x = 0
            
        # Z boundaries
        if agent.position.z < 0:
            agent.position.z = 0
            agent.velocity.z = 0
        elif agent.position.z > self.depth:
            agent.position.z = self.depth
            agent.velocity.z = 0
        
        # Check collision with obstacles
        for obstacle in self.obstacles:
            if obstacle.contains_point(agent.position.x, agent.position.z):
                # Calculate the closest edge to push back to
                dx_left = agent.position.x - obstacle.x
                dx_right = (obstacle.x + obstacle.width) - agent.position.x
                dz_front = agent.position.z - obstacle.z
                dz_back = (obstacle.z + obstacle.depth) - agent.position.z
                
                # Find closest edge
                min_dist = min(dx_left, dx_right, dz_front, dz_back)
                
                # Push agent back to closest edge
                if min_dist == dx_left:
                    agent.position.x = obstacle.x
                    agent.velocity.x = 0
                elif min_dist == dx_right:
                    agent.position.x = obstacle.x + obstacle.width
                    agent.velocity.x = 0
                elif min_dist == dz_front:
                    agent.position.z = obstacle.z
                    agent.velocity.z = 0
                elif min_dist == dz_back:
                    agent.position.z = obstacle.z + obstacle.depth
                    agent.velocity.z = 0

def _make_decision(self):
    """Use neural network to decide actions."""
    # Calculate distance from home
    distance_from_home = ((self.position.x - self.home_position.x)**2 + 
                         (self.position.z - self.home_position.z)**2)**0.5
    
    # Check for nearby obstacles
    closest_obstacle_dist = float('inf')
    obstacle_direction_x = 0
    obstacle_direction_z = 0
    
    for obstacle in self.world.obstacles:
        # Calculate distances to obstacle edges
        dist_to_left = abs(self.position.x - obstacle.x)
        dist_to_right = abs(self.position.x - (obstacle.x + obstacle.width))
        dist_to_front = abs(self.position.z - obstacle.z)
        dist_to_back = abs(self.position.z - (obstacle.z + obstacle.depth))
        
        # Find closest edge
        min_dist = min(dist_to_left, dist_to_right, dist_to_front, dist_to_back)
        
        if min_dist < closest_obstacle_dist:
            closest_obstacle_dist = min_dist
            
            # Calculate direction vector to obstacle
            if min_dist == dist_to_left:
                obstacle_direction_x = -1  # Obstacle is to the left
                obstacle_direction_z = 0
            elif min_dist == dist_to_right:
                obstacle_direction_x = 1   # Obstacle is to the right
                obstacle_direction_z = 0
            elif min_dist == dist_to_front:
                obstacle_direction_x = 0
                obstacle_direction_z = -1  # Obstacle is in front
            elif min_dist == dist_to_back:
                obstacle_direction_x = 0
                obstacle_direction_z = 1   # Obstacle is behind
    
    # Cap the sensing distance
    closest_obstacle_dist = min(closest_obstacle_dist, 10.0)
    
    # Normalize distance for network input
    if closest_obstacle_dist < 10.0:
        obstacle_dist_normalized = 1.0 - (closest_obstacle_dist / 10.0)
    else:
        obstacle_dist_normalized = 0.0
    
    # Prepare inputs for the neural network
    inputs = [
        self.position.x / 50.0,                 # Normalized position
        self.position.z / 50.0,
        self.velocity.x / self.max_speed,       # Normalized velocity
        self.velocity.z / self.max_speed,
        distance_from_home / 30.0,              # Normalized distance from home
        1.0 if self.grounded else 0.0,          # Grounded state
        obstacle_dist_normalized,               # Distance to nearest obstacle
        obstacle_direction_x,                   # Direction to nearest obstacle
        obstacle_direction_z
    ]
    
    # Process through neural network and make decision
    # (rest of your decision code remains the same)





In [30]:
# Update the neural network initialization
def __init__(self, x=0.0, y=0.0, z=0.0, mass=70.0):
    super().__init__(x, y, z, mass)
    
    # Agent properties
    self.name = f"Neural-{id(self) % 1000}"
    self.decision_cooldown = 0
    self.decision_interval = 10  # Make decisions every N steps
    self.home_position = Vector3(x, y, z)
    
    # Neural network setup - now with obstacle sensing
    self.input_size = 9  # Position x,z, velocity x,z, distance from home, grounded, obstacle distance, obstacle direction x,z
    self.hidden_size = 12  # Increased for more complex behavior
    self.output_size = 5  # N, E, S, W, Jump
    self.brain = NeuralNetwork(self.input_size, self.hidden_size, self.output_size)

def calculate_fitness(self):
    """Calculate fitness score for evolution."""
    # Base fitness from distance traveled
    fitness = self.distance_traveled
    
    # Bonus for jumps
    fitness += self.jumps_performed * 2.0
    
    # Bonus for no collisions
    fitness += 50.0 * (1.0 - min(1.0, self.collision_count / 10.0))
    
    # More rewards for getting farther from starting point
    current_distance = ((self.position.x - self.start_position.x)**2 + 
                        (self.position.z - self.start_position.z)**2)**0.5
    fitness += current_distance * 0.5
    
    self.fitness = fitness
    return fitness



In [31]:
def visualize_world_with_obstacles(world, cell_size=5):
    """Create a text-based visualization of the world with obstacles."""
    width_cells = int(world.width / cell_size)
    depth_cells = int(world.depth / cell_size)
    
    # Create empty grid
    grid = [[' ' for _ in range(width_cells)] for _ in range(depth_cells)]
    
    # Mark boundaries
    for x in range(width_cells):
        grid[0][x] = '#'
        grid[depth_cells-1][x] = '#'
    for z in range(depth_cells):
        grid[z][0] = '#'
        grid[z][width_cells-1] = '#'
    
    # Mark obstacles
    for obstacle in world.obstacles:
        x_start = max(0, min(width_cells-1, int(obstacle.x / cell_size)))
        x_end = max(0, min(width_cells-1, int((obstacle.x + obstacle.width) / cell_size)))
        z_start = max(0, min(depth_cells-1, int(obstacle.z / cell_size)))
        z_end = max(0, min(depth_cells-1, int((obstacle.z + obstacle.depth) / cell_size)))
        
        for z in range(z_start, z_end + 1):
            for x in range(x_start, x_end + 1):
                grid[z][x] = 'X'
    
    # Place agents
    for agent in world.agents:
        grid_x = min(width_cells-1, max(0, int(agent.position.x / cell_size)))
        grid_z = min(depth_cells-1, max(0, int(agent.position.z / cell_size)))
        
        # Only overwrite if not an obstacle
        if grid[grid_z][grid_x] != 'X':
            grid[grid_z][grid_x] = 'A'
    
    # Print the grid
    print("\nWorld visualization:")
    for row in grid:
        print(''.join(row))
    print("")

In [33]:
import random

def create_world_with_obstacles(num_obstacles=10, world_size=100.0):
    """Create a world with randomly placed obstacles."""
    world = World(width=world_size, depth=world_size)
    
    # Add obstacles
    for i in range(num_obstacles):
        # Random position
        x = random.uniform(5, world_size - 15)
        z = random.uniform(5, world_size - 15)
        
        # Random size
        width = random.uniform(3, 10)
        depth = random.uniform(3, 10)
        
        # Add the obstacle
        world.add_obstacle(x, 0, z, width, depth)
    
    return world

In [34]:
def run_evolution_with_obstacles(generations=10, steps_per_generation=500, population_size=15):
    """Run the neural network evolution simulation with obstacles."""
    # Create a world with obstacles
    world = create_world_with_obstacles(num_obstacles=8, world_size=100.0)
    
    # Initialize evolution simulation
    sim = EvolutionSimulation(population_size=population_size)
    sim.world = world  # Use our custom world with obstacles
    
    # Initialize first generation
    sim.initialize()
    
    # Visualize initial world state
    visualize_world_with_obstacles(sim.world)
    
    for gen in range(generations):
        print(f"\nStarting Generation {gen+1}")
        fitnesses = sim.run_generation(steps=steps_per_generation)
        
        # Visualize world after generation
        if gen % 2 == 0:  # Visualize every other generation
            visualize_world_with_obstacles(sim.world)
        
        # Create next generation
        sim.create_next_generation(fitnesses)
        
        # Print best agent info
        best_agent = fitnesses[0][0]
        print(f"Best agent in gen {gen+1}: {best_agent.name}, " +
              f"Fitness: {best_agent.fitness:.2f}, " +
              f"Distance: {best_agent.distance_traveled:.2f}, " +
              f"Jumps: {best_agent.jumps_performed}")
    
    print("\nEvolution complete!")
    print(f"Best fitness achieved: {sim.best_fitness:.2f}")
    
    return sim

# Run the evolution
evolution_sim = run_evolution_with_obstacles(generations=5, steps_per_generation=300)


Generation 1 initialized with 15 agents

World visualization:
####################
#                  #
#      A           #
#   A  A    A    A #
#  A     A         #
#          A       #
#               A  #
#      A           #
#                  #
#   A              #
#                  #
#                  #
#                  #
#                  #
#        A   A     #
#        A         #
#     A            #
#                  #
#                  #
####################


Starting Generation 1
Running generation 1 for 300 steps...


AttributeError: 'World' object has no attribute 'update'